# Generate Data

In [58]:
import numpy as np
import math
from doubleml.datasets import make_plr_CCDDHNR2018, make_plr_turrell2018
np.random.seed(1234)
n_rep = 1
n_obs = 10000
n_vars = 10
alpha = 0.5
data = list()
from sklearn.datasets import make_spd_matrix

def g(x):
    return np.exp(x)

def m(x):
    return np.sin(x)

theta = alpha = 0.5 
b = [1/k for k in range(1,n_vars+1)] # x weights 
sigma = make_spd_matrix(n_vars, random_state=42)

for i_rep in range(n_rep):
    import pandas as pd
    import numpy as np
    df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/smoking.csv')
    cat = df.select_dtypes('object').columns
    df = pd.get_dummies(df, columns = cat, drop_first = True)
    outcome = 'bweight'
    treatment = 'mbsmoke_Smoker'
    rest = list(df.drop([outcome, treatment,], axis = 1).columns)
    rest = ['mhisp', 'fhisp', 'foreign', 'alcohol', 'deadkids', 'mage',
       'medu', 'fage', 'fedu', 'nprenatal', 'monthslb', 'order', 'mrace',
       'frace', 'prenatal', 'birthmonth', 'lbweight', 'mmarried_Not married', 'fbaby_Yes', 'prenatal1_Yes']
    rest = ['mhisp', 'fhisp', 'foreign', 'alcohol', 'deadkids', 'fage', 'fedu', 'nprenatal', 'monthslb', 'order', 'mrace',]

    df = df[[outcome] + [treatment] + rest]
    #import wooldridge
    #df = wooldridge.data('jtrain3')
    #df['avg'] = 0.5 * (df.re74+df.re75)
    #df = df.dropna()
    df = df.fillna(0)
    #df = df[df.avg <= 15]
    y = df[outcome]
    d = df[treatment]
    x = df[rest].astype('float')
    print(y.shape, x.shape, d.shape)
    x.head()
    data.append((x, y, d))

(4642,) (4642, 11) (4642,)


In [63]:
from sklearn.linear_model import LinearRegression, LogisticRegression
OLS_Y = LinearRegression()
OLS_D = LogisticRegression()
RF_Y = RandomForestRegressor(n_estimators = 5)
RF_D = RandomForestClassifier(n_estimators = 5)

from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.086 | 0.671 |
| D on X (Accuracy) | 0.816 | 0.941 |
+-------------------+-------+-------+


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Naive ML
- no orthogonalisation, no crossfitting

In [64]:
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, y - g_hat)
    return psi_a, psi_b

In [65]:
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')
np.random.seed(1111)
ml_l = RandomForestRegressor()
ml_m = RandomForestClassifier()
#ml_m = LogisticRegression()
ml_g = clone(ml_l)

# to speed up the illustration we hard-code the simulation results
theta_nonorth = np.empty(n_rep)
se_nonorth = np.empty(n_rep)
t_nonorth = np.empty(n_rep)
p_nonorth = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y.ravel(), d.ravel())
    print(obj_dml_data)
    obj_dml_plr_nonorth = DoubleMLPLR(obj_dml_data,
                                      ml_l, ml_m, ml_g,
                                      n_folds=1,
                                      apply_cross_fitting=False,
                                      score=non_orth_score)
    obj_dml_plr_nonorth.fit()
    theta_nonorth[i_rep] = obj_dml_plr_nonorth.coef[0]
    se_nonorth[i_rep] = obj_dml_plr_nonorth.se[0]

0
================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11']
Instrument variable(s): None
No. Observations: 4642

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4642 entries, 0 to 4641
Columns: 13 entries, X1 to d
dtypes: float64(13)
memory usage: 471.6 KB



# Orthogonal Machine Learning
- resolves regularization bias, but not overfitting

In [66]:
import numpy as np
np.random.seed(2222)
# to speed up the illustration we hard-code the simulation results
theta_orth_nosplit = np.empty(n_rep)
se_orth_nosplit = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr_orth_nosplit = DoubleMLPLR(obj_dml_data,
                                           ml_l, ml_m, ml_g,
                                           n_folds=1,
                                           score='IV-type',
                                           apply_cross_fitting=False)
    obj_dml_plr_orth_nosplit.fit()
    theta_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.coef[0]
    se_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.se[0]

0


# Orthogonal ML + Cross fitting (DML)

In [67]:
np.random.seed(42)
# to speed up the illustration we hard-code the simulation results
theta_dml = np.empty(n_rep)
se_dml = np.empty(n_rep)

# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                              ml_l, ml_m, ml_g,
                              n_folds=2,
                              score='IV-type')
    obj_dml_plr.fit()
    theta_dml[i_rep] = obj_dml_plr.coef[0]
    se_dml[i_rep] = obj_dml_plr.se[0]

0


# Regular OLS

In [68]:
np.random.seed(42)
import statsmodels.api as sm # for OLS 
# to speed up the illustration we hard-code the simulation results
theta_ols = np.empty(n_rep)
se_ols = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    (x, y, d) = data[i_rep]
    OLS = sm.OLS(y,sm.add_constant(np.c_[d,x]))
    results = OLS.fit()
    theta_ols[i_rep] = results.params[1]
    se_ols[i_rep] = results.bse[1]   

# Distribution of Theta

In [69]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]

table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 's.e(θ_hat)','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[results.params[1], results.bse[1], results.tvalues[1], results.pvalues[1], results.conf_int(alpha=0.05, cols=None)[0][1], results.conf_int(alpha=0.05, cols=None)[1][1]].reshape(-1).tolist()
table.add_row(a)
a = ['Naive-ML']+ np.array(obj_dml_plr_nonorth.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Ortho-ML']+ np.array(obj_dml_plr_orth_nosplit.summary).reshape(-1).tolist()
table.add_row(a)
a = ['OrthoML+Crossfitting (DML)']+ np.array(obj_dml_plr.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------+----------+------------+---------+-------+----------+----------+
|         Estimator          |  θ_hat   | s.e(θ_hat) |    t    |   p   |   2.5%   |  97.25%  |
+----------------------------+----------+------------+---------+-------+----------+----------+
|            OLS             | -233.953 |   21.590   | -10.836 | 0.000 | -276.280 | -191.626 |
|          Naive-ML          | -218.479 |   8.937    | -24.448 | 0.000 | -235.995 | -200.964 |
|          Ortho-ML          | -223.522 |   15.243   | -14.664 | 0.000 | -253.399 | -193.646 |
| OrthoML+Crossfitting (DML) | -220.449 |   25.128   |  -8.773 | 0.000 | -269.700 | -171.198 |
+----------------------------+----------+------------+---------+-------+----------+----------+


# First Stage Checks

In [70]:
from sklearn.linear_model import LinearRegression, LogisticRegression
OLS_Y = LinearRegression()
OLS_D = LogisticRegression()
RF_Y = RandomForestRegressor()
RF_D = RandomForestClassifier()

from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.086 | 0.773 |
| D on X (Accuracy) | 0.816 | 0.970 |
+-------------------+-------+-------+


In [71]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]
table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.086 | 0.770 |
| D on X (Accuracy) | 0.816 | 0.970 |
+-------------------+-------+-------+
